In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms

import time
import numpy as np
import i8ie

def log_softmax(x):
    e_x = np.exp(x[:,] - np.max(x[:,]))
    return np.log(e_x / e_x.sum())

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(784, 10)
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

model = Net()
model.load_state_dict(torch.load('model28.pt'))

batch_size = 10000
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ])
test_dataset = torchvision.datasets.MNIST(root='./data/mnist/', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

xs = []
ts = []
for batch_idx, (x, target) in enumerate(test_loader):
    xs.append(x)
    ts.append(target)

In [2]:
w = model.state_dict()['fc.weight'].numpy()
b = model.state_dict()['fc.bias'].numpy()

In [3]:
%%time
correct_cnt = 0
for x,target in zip(xs,ts):
    x = x.reshape(-1,784).numpy()
    x = i8ie.qfc(w,b,x)
    p = np.argmax(log_softmax(x), axis = 1)
    correct_cnt += (p == target.numpy()).sum()
    
print(correct_cnt)

9134
CPU times: user 17.2 ms, sys: 1.25 ms, total: 18.5 ms
Wall time: 5.74 ms


In [4]:
%%time
correct_cnt, ave_loss = 0, 0
for x,target in zip(xs,ts):
    out = model(x)
    _, pred_label = torch.max(out.data, 1)
    correct_cnt += (pred_label == target.data).sum()
print(correct_cnt)

tensor(9250)
CPU times: user 18.2 ms, sys: 0 ns, total: 18.2 ms
Wall time: 2.27 ms
